In [12]:
pip install --upgrade pinecone

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import uuid


In [3]:
pc = Pinecone( api_key = "pcsk_77xAnA_9TqB8jqEyZpBhzvPZDuda4qHRughvRH8D94hBL4evJknJUM4FJyJtpi5Z4Gc8hk")

In [6]:
index_name = "semantic-search"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension = 384,
        metric = "cosine",
        spec = ServerlessSpec(
             cloud="aws",
            region="us-east-1"
        )
    )
index = pc.Index(index_name)
print("Index is Ready")

Index is Ready


In [4]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [7]:
documents = [
    "Flask is a lightweight Python web framework.",
    "Transformers use attention mechanisms in NLP.",
    "Pinecone is a vector database for semantic search.",
    "Quantization reduces model size and speeds up inference.",
    "Vector databases store embeddings for similarity search."
]

vectors = []

for doc in documents:
    embedding = model.encode(doc).tolist()
    vectors.append(
        (str(uuid.uuid4()), embedding, {"text":doc})
    )

index.upsert(vectors)
print("Document indexed successfully")

Document indexed successfully


Semantic Search Function

In [10]:
def semantic_search(query, top_k=2):
    query_embedding = model.encode(query).tolist()

    result = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )

    return [match["metadata"]["text"] for match in result["matches"]]


In [11]:
print("\n🔍 Semantic Search Engine (type 'exit' to quit)")

while True:
    query  = input("\nQuery:    ")
    if query.lower()=="exit":
        break

    results = semantic_search(query)
    print("\nResults:   ")
    for r in results:
        print("-",r)


🔍 Semantic Search Engine (type 'exit' to quit)

Results:   
- Flask is a lightweight Python web framework.
- Pinecone is a vector database for semantic search.

Results:   
- Vector databases store embeddings for similarity search.
- Pinecone is a vector database for semantic search.
